# Устанавливаем библиотеки

In [ ]:
!pip install torch
!pip install dlib
!pip install face-alignment

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    drive = None
    print("You must run the code in colab")
    pass

In [ ]:
from collections import OrderedDict

import face_alignment
import cv2
import numpy as np
from glob import glob
from pathlib import PurePath, Path
from matplotlib import pyplot as plt

from converter.config import SegmentationType

Устанавливаем глобальные константы

In [ ]:
DIR_TRAIN = "/content/drive/MyDrive/faceswap_train"
DIR_FACE_A = f"{DIR_TRAIN}/face_src/rgb"
DIR_FACE_B = f"{DIR_TRAIN}/face_dst/rgb"
DIR_BM_FACE_A_EYES = f"{DIR_TRAIN}/binary_masks/faceA_eyes"
DIR_BM_FACE_B_EYES = f"{DIR_TRAIN}/binary_masks/faceB_eyes"


In [ ]:
FNS_FACE_A = glob(f"{DIR_FACE_A}/*.*")
FNS_FACE_B = glob(f"{DIR_FACE_B}/*.*")

Определяем нужные пределы для частей лица

In [ ]:
# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("jaw", (0, 17)),
    ("eyebrow_r", (17, 22)),
    ("eyebrow_l", (22, 27)),
    ("nose", (27, 35)),
    ("eye_r", (36, 42)),
    ("eye_l", (42, 48)),
    ("mouth", (48, 68))
])

In [ ]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cuda', flip_input=False)

Очищаем папки и заново создаем

In [ ]:
% rm -rf / content / drive / MyDrive / faceswap_train / binary_masks / faceA_eyes
% rm -rf / content / drive / MyDrive / faceswap_train / binary_masks / faceB_eyes

Path(DIR_BM_FACE_A_EYES).mkdir(parents=True, exist_ok=True)
Path(DIR_BM_FACE_B_EYES).mkdir(parents=True, exist_ok=True)

In [ ]:
eye_r_idx = FACIAL_LANDMARKS_IDXS['eye_r']
eye_l_idx = FACIAL_LANDMARKS_IDXS['eye_l']
nose_idx = FACIAL_LANDMARKS_IDXS['nose']
mouth_idx = FACIAL_LANDMARKS_IDXS['mouth']

def lm_range(t: tuple):
    return range(t[0], t[1])


def draw_mask(predictions, mask, pnts):
    pnts = [(predictions[i, 0], predictions[i, 1])
            for i in range(pnts[0], pnts[1])]
    hull = cv2.convexHull(np.array(pnts)).astype(np.int32)
    return cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)


def face_segmentation(image, preds, option = SegmentationType.ALL):
    mask = np.zeros_like(image)
    if option == SegmentationType.EYES_ONLY:
        mask = draw_mask(preds, mask, eye_r_idx)
        mask = draw_mask(preds, mask, eye_l_idx)
        pass
    elif option == SegmentationType.NOSE_ONLY:
        mask = draw_mask(preds, mask, nose_idx)
        pass
    elif option == SegmentationType.MOUTH_ONLY:
        mask = draw_mask(preds, mask, mouth_idx)
        pass
    elif option == SegmentationType.ALL:
        mask = draw_mask(preds, mask, eye_r_idx)  # Draw right eye binary mask
        mask = draw_mask(preds, mask, eye_l_idx)  # Draw left eye binary mask
        mask = draw_mask(preds, mask, nose_idx)  # Draw nose binary mask
        mask = draw_mask(preds, mask, mouth_idx)  # Draw mouth binary mask
        pass
    mask = cv2.dilate(mask, np.ones((13, 13), np.uint8), iterations=1)
    mask = cv2.GaussianBlur(mask, (7, 7), 0)

    return mask

In [ ]:
seg_type = SegmentationType.ALL
fns_face_not_detected = []

In [ ]:
for idx, fns in enumerate([FNS_FACE_A, FNS_FACE_B]):
    save_path = DIR_BM_FACE_A_EYES if idx == 0 else DIR_BM_FACE_B_EYES

    # create binary mask for each training image
    for fn in fns:
        raw_fn = PurePath(fn).parts[-1]

        x = cv2.imread(fn)
        x = cv2.resize(x, (256, 256))
        preds = fa.get_landmarks(x)
        mask = np.zeros_like(x)

        if preds is not None:
            preds = preds[0]
            mask = face_segmentation(seg_type, preds)
            pass
        else:
            print(f"No faces were detected in image '{fn}'")
            fns_face_not_detected.append(fn)
            pass

        plt.imsave(fname=f"{save_path}/{raw_fn}", arr=mask, format="png")
        pass

In [ ]:
num_faceA = len(glob(f"{DIR_FACE_A}/*.*"))
num_faceB = len(glob(f"{DIR_FACE_B}/*.*"))

print("Nuber of processed images: " + str(num_faceA + num_faceB))
print("Number of image(s) with no face detected: " + str(len(fns_face_not_detected)))

# Случайно показываем лицо и двоичную маску

In [ ]:
face = np.random.choice(["A", "B"])

dir_face = DIR_FACE_A if face == "A" else DIR_FACE_B
fns_face = FNS_FACE_A if face == "A" else FNS_FACE_B
num_face = len(glob(dir_face + "/*.*"))
rand_idx = np.random.randint(num_face)
rand_fn = fns_face[rand_idx]
raw_fn = PurePath(rand_fn).parts[-1]
mask_fn = f"{DIR_BM_FACE_A_EYES}/{raw_fn}" if face == "A" else f"{DIR_BM_FACE_B_EYES}/{raw_fn}"
resize_shape = (256, 256)

im = cv2.imread(rand_fn)
mask = cv2.imread(mask_fn)

# im = cv2.resize(im, resize_shape)
# mask = cv2.resize(mask, resize_shape)

if rand_fn in fns_face_not_detected:
    print("========== На этом изображении не было обнаружено никаких лиц! ==========")

fig = plt.figure(figsize=(15, 6))
plt.subplot(1, 3, 1)
plt.grid('off')
plt.imshow(im)
plt.subplot(1, 3, 2)
plt.grid('off')
plt.imshow(mask)
plt.subplot(1, 3, 3)
plt.grid('off')
plt.imshow((mask / 255 * im).astype(np.uint8))

# Случайно отображать изображения, на котором не обнаружено лица.

In [ ]:
num_no_face_img = len(fns_face_not_detected)
rand_idx = np.random.randint(num_no_face_img)
x = plt.imread(fns_face_not_detected[rand_idx])
plt.grid('off')
plt.imshow(x)